In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 21066 (delta 28), reused 14 (delta 6), pack-reused 21008
Receiving objects: 100% (21066/21066), 16.53 MiB | 26.13 MiB/s, done.
Resolving deltas: 100% (15385/15385), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728),

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
import os

Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import math
from math import radians
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [4]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 3
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [5]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [6]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [7]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [8]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [9]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [10]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [11]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [12]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [13]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [14]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [15]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0.09:
        dataset.iloc[i,j] = 0
  return dataset

In [16]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2.rename(columns = {'T' : 'Temp'}, inplace = True)
dataset2

,DHI,DNI,WS,RH,Temp,TARGET,Time,Date
0,0,0,1.5,69.08,-12,0.0,0.0,0
1,0,0,1.5,69.06,-12,0.0,0.5,0
2,0,0,1.6,71.78,-12,0.0,1.0,0
3,0,0,1.6,71.75,-12,0.0,1.5,0
4,0,0,1.6,75.20,-12,0.0,2.0,0
...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364
52556,0,0,2.4,66.79,-4,0.0,22.0,364
52557,0,0,2.2,66.78,-4,0.0,22.5,364
52558,0,0,2.1,67.72,-4,0.0,23.0,364


In [17]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,-0.258819,0.965926,0,23.0


In [18]:
dataset5= dataset4.copy()

dataset5['GHIs'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [19]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs
50,0,0,1.6,88.11,-11,0.0,0.258819,0.965926,23,1.0,0.0


In [20]:
def add_variables(dataset):
  sunny = list()
  avg_temp = list()
  max_temp = list()
  min_temp = list()
  avg_ghi = list()
  max_ghi = list()
  min_ghi = list()
  avg_tar = list()
  max_tar = list()
  min_tar = list()


  for i in range(0, len(dataset), 48):
    temp = dataset['Temp'][i:i+48]
    ghi = dataset['GHIs'][i:i+48]
    tar = dataset['TARGET'][i:i+48]
    sunny += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 
    avg_temp += [temp.mean()]*48
    max_temp += [temp.max()]*48
    min_temp += [temp.min()]*48
    avg_ghi += [ghi.sum()/(24-(list(dataset.DHI[i:i+48]).count(0)*0.5))]*48
    max_ghi += [ghi.max()]*48
    min_ghi += [ghi[ghi>0.01].min()]*48
    avg_tar += [tar.sum()/(24-(list(dataset.DHI[i:i+48]).count(0)*0.5))]*48
    max_tar += [tar.max()]*48
    min_tar += [tar[tar>0.01].min()]*48


  dataset["SH"] = sunny
  dataset["Avg_temp"] = avg_temp
  dataset['Max_temp'] = max_temp
  dataset['Min_temp'] = min_temp
  dataset["Avg_GHI"] = avg_ghi
  dataset['Max_GHI'] = max_ghi
  dataset['Min_GHI'] = min_ghi
  dataset["Avg_TAR"] = avg_tar
  dataset['Max_TAR'] = max_tar
  dataset['Min_TAR'] = min_tar
  
  return dataset

In [21]:
dataset6 = add_variables(dataset5)
dataset6

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs,SH,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,Avg_TAR,Max_TAR,Min_TAR
0,0,0,1.5,69.08,-12,0.0,0.000000,1.000000,23,0.0,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
1,0,0,1.5,69.06,-12,0.0,0.130526,0.991445,23,0.5,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
2,0,0,1.6,71.78,-12,0.0,0.258819,0.965926,23,1.0,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
3,0,0,1.6,71.75,-12,0.0,0.382683,0.923880,23,1.5,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
4,0,0,1.6,75.20,-12,0.0,0.500000,0.866025,23,2.0,0.0,9.0,-7.979167,-3,-12,578.031164,705.717934,10.000000,34.777337,33.129393,0.938541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,-0.608761,0.793353,0,21.5,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448
52556,0,0,2.4,66.79,-4,0.0,-0.500000,0.866025,0,22.0,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448
52557,0,0,2.2,66.78,-4,0.0,-0.382683,0.923880,0,22.5,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448
52558,0,0,2.1,67.72,-4,0.0,-0.258819,0.965926,0,23.0,0.0,9.0,-2.708333,1,-5,1024.965465,771.947011,169.653975,46.454798,39.322261,2.252448


In [22]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'Temp','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset7 = dataset6[['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'SH', 'sin_time', 'cos_time', 'GHIs', 'Avg_TAR', 'Max_TAR', 'Min_TAR', 'TARGET']]
dataset7.head(10)

,RH,Temp,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,SH,sin_time,cos_time,GHIs,Avg_TAR,Max_TAR,Min_TAR,TARGET
0,69.08,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.000000,1.000000,0.0,34.777337,33.129393,0.938541,0.0
1,69.06,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.130526,0.991445,0.0,34.777337,33.129393,0.938541,0.0
2,71.78,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.258819,0.965926,0.0,34.777337,33.129393,0.938541,0.0
3,71.75,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.382683,0.923880,0.0,34.777337,33.129393,0.938541,0.0
4,75.20,-12,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.500000,0.866025,0.0,34.777337,33.129393,0.938541,0.0
5,69.29,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.608761,0.793353,0.0,34.777337,33.129393,0.938541,0.0
6,72.56,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.707107,0.707107,0.0,34.777337,33.129393,0.938541,0.0
7,72.55,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.793353,0.608761,0.0,34.777337,33.129393,0.938541,0.0
8,74.62,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.866025,0.500000,0.0,34.777337,33.129393,0.938541,0.0
9,74.61,-11,-7.979167,-3,-12,578.031164,705.717934,10.0,9.0,0.923880,0.382683,0.0,34.777337,33.129393,0.938541,0.0


In [23]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(dataset7)
temp_X = pd.DataFrame(scaler.transform(dataset7), columns = dataset7.columns)
temp_y = pd.DataFrame(dataset7['TARGET'])

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [24]:
dataset8 = train_to_supervised(temp_X,temp_y, n_days)
dataset8

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),...,cos_time(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,0.665404,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.500000,1.000000,0.0,0.232124,0.254563,0.096779,0.0,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.565263,0.995722,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,0.961940,0.0,0.071674,0.078569,0.075271,0.0,0.795368,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.083333,0.370590,0.982963,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.083333,0.434737,0.995722,0.0,0.071674,0.078569,0.075271,0.0,0.0,0.0
145,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.565263,0.995722,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.629410,0.982963,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,0.982963,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.083333,0.434737,0.995722,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.500000,1.000000,0.0,0.198174,0.309021,0.032260,0.0,0.0,0.0
146,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.629410,0.982963,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.691342,0.961940,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,0.995722,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.500000,1.000000,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.565263,0.995722,0.0,0.198174,0.309021,0.032260,0.0,0.0,0.0
147,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.691342,0.961940,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.750000,0.933013,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,1.000000,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.565263,0.995722,0.0,0.198174,0.309021,0.032260,0.0,0.758576,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.629410,0.982963,0.0,0.198174,0.309021,0.032260,0.0,0.0,0.0
148,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.750000,0.933013,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.804381,0.896677,0.0,0.232124,0.25456

In [25]:
X = dataset8.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)
y = pd.DataFrame(dataset8.iloc[:,-2:])
X

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),...,SH(t-3),sin_time(t-3),cos_time(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
144,0.665404,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.500000,1.000000,0.0,0.232124,0.254563,0.096779,0.0,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.565263,0.995722,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,0.083333,0.308658,0.961940,0.0,0.071674,0.078569,0.075271,0.0,0.795368,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.083333,0.370590,0.982963,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.083333,0.434737,0.995722,0.0,0.071674,0.078569,0.075271,0.0
145,0.665188,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.565263,0.995722,0.0,0.232124,0.254563,0.096779,0.0,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.629410,0.982963,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,0.083333,0.370590,0.982963,0.0,0.071674,0.078569,0.075271,0.0,0.795260,0.111111,0.191123,0.239130,0.162162,0.100076,0.373454,0.018912,0.083333,0.434737,0.995722,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.500000,1.000000,0.0,0.198174,0.309021,0.032260,0.0
146,0.694622,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.629410,0.982963,0.0,0.232124,0.254563,0.096779,0.0,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.691342,0.961940,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,0.083333,0.434737,0.995722,0.0,0.071674,0.078569,0.075271,0.0,0.759333,0.092593,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.500000,1.000000,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.565263,0.995722,0.0,0.198174,0.309021,0.032260,0.0
147,0.694297,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.691342,0.961940,0.0,0.232124,0.254563,0.096779,0.0,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.750000,0.933013,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,...,0.083333,0.500000,1.000000,0.0,0.198174,0.309021,0.032260,0.0,0.759442,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.565263,0.995722,0.0,0.198174,0.309021,0.032260,0.0,0.758576,0.111111,0.210966,0.260870,0.135135,0.262510,0.790891,0.008105,0.083333,0.629410,0.982963,0.0,0.198174,0.309021,0.032260,0.0
148,0.731631,0.129630,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.750000,0.933013,0.0,0.232124,0.254563,0.096779,0.0,0.667677,0.148148,0.153525,0.173913,0.189189,0.299808,0.585426,0.024316,0.083333,0.80

In [26]:
X_train, X_valid, Y_train,  Y_valid = train_test_split(X, y, test_size=0.30, random_state=42)

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [27]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [28]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [29]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season', 'T' : 'Temp'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,-0.258819,0.965926,22,23.0


In [30]:
testset5 = testset4.copy()
testset5['GHIs'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [31]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs
50,0,0,2.1,61.33,-10.8,0.0,0.258819,0.965926,22,1.0,0.0


In [32]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,Temp,TARGET,sin_time,cos_time,season,Time,GHIs,SH,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,Avg_TAR,Max_TAR,Min_TAR
50,0,0,1.6,88.11,-11,0.0,0.258819,0.965926,23,1.0,0.0,9.0,-6.3125,2,-12,1466.243756,1036.281607,23.0,58.800983,48.612666,2.158549


In [33]:
testset6 = add_variables(testset5)

In [34]:
testset6 = testset5[['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'SH', 'sin_time', 'cos_time', 'GHIs', 'Avg_TAR', 'Max_TAR', 'Min_TAR', 'TARGET']]
testset6.head(10)

,RH,Temp,Avg_temp,Max_temp,Min_temp,Avg_GHI,Max_GHI,Min_GHI,SH,sin_time,cos_time,GHIs,Avg_TAR,Max_TAR,Min_TAR,TARGET
0,70.34,-6.1,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.000000,1.000000,0.0,36.177302,40.637337,0.656981,0.0
1,71.97,-6.4,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.130526,0.991445,0.0,36.177302,40.637337,0.656981,0.0
2,69.77,-6.7,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.258819,0.965926,0.0,36.177302,40.637337,0.656981,0.0
3,71.95,-7.1,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.382683,0.923880,0.0,36.177302,40.637337,0.656981,0.0
4,69.48,-7.4,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.500000,0.866025,0.0,36.177302,40.637337,0.656981,0.0
5,72.13,-7.9,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.608761,0.793353,0.0,36.177302,40.637337,0.656981,0.0
6,69.76,-8.3,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.707107,0.707107,0.0,36.177302,40.637337,0.656981,0.0
7,71.97,-8.7,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.793353,0.608761,0.0,36.177302,40.637337,0.656981,0.0
8,68.56,-9.1,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.866025,0.500000,0.0,36.177302,40.637337,0.656981,0.0
9,70.75,-9.5,-7.3875,-2.1,-10.7,654.572846,821.168341,7.0,9.5,0.923880,0.382683,0.0,36.177302,40.637337,0.656981,0.0


In [35]:
testset6 = pd.DataFrame(scaler.transform(testset6), columns = testset6.columns)

In [36]:
testset7 = test_to_supervised(testset6, n_days)
testset7

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),...,SH(t-3),sin_time(t-3),cos_time(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
144,0.679039,0.238889,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.500000,1.000000,0.0,0.244677,0.338365,0.064521,0.0,0.696678,0.233333,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.565263,0.995722,0.0,0.244677,0.338365,0.064521,0.0,0.672871,0.227778,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,...,0.166667,0.308658,0.961940,0.0,0.434665,0.421102,0.150552,0.0,0.499188,0.270370,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.166667,0.370590,0.982963,0.0,0.434665,0.421102,0.150552,0.0,0.494752,0.272222,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.166667,0.434737,0.995722,0.0,0.434665,0.421102,0.150552,0.0
145,0.696678,0.233333,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.565263,0.995722,0.0,0.244677,0.338365,0.064521,0.0,0.672871,0.227778,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.629410,0.982963,0.0,0.244677,0.338365,0.064521,0.0,0.696461,0.220370,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,...,0.166667,0.370590,0.982963,0.0,0.434665,0.421102,0.150552,0.0,0.494752,0.272222,0.199321,0.265217,0.167568,0.877605,0.940365,0.287738,0.166667,0.434737,0.995722,0.0,0.434665,0.421102,0.150552,0.0,0.512282,0.274074,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.500000,1.000000,0.0,0.288665,0.315278,0.139784,0.0
146,0.672871,0.227778,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.629410,0.982963,0.0,0.244677,0.338365,0.064521,0.0,0.696461,0.220370,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.691342,0.961940,0.0,0.244677,0.338365,0.064521,0.0,0.669733,0.214815,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,...,0.166667,0.434737,0.995722,0.0,0.434665,0.421102,0.150552,0.0,0.512282,0.274074,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.500000,1.000000,0.0,0.288665,0.315278,0.139784,0.0,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.565263,0.995722,0.0,0.288665,0.315278,0.139784,0.0
147,0.696461,0.220370,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.691342,0.961940,0.0,0.244677,0.338365,0.064521,0.0,0.669733,0.214815,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.750000,0.933013,0.0,0.244677,0.338365,0.064521,0.0,0.698409,0.205556,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,...,0.166667,0.500000,1.000000,0.0,0.288665,0.315278,0.139784,0.0,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.565263,0.995722,0.0,0.288665,0.315278,0.139784,0.0,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.629410,0.982963,0.0,0.288665,0.315278,0.139784,0.0
148,0.669733,0.214815,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.750000,0.933013,0.0,0.244677,0.338365,0.064521,0.0,0.698409,0.205556,0.168355,0.193478,0.224324,0.347313,0.698882,0.016211,0.166667,0.80

In [37]:
testset9 = testset7.iloc[-48:,:] #season을 다시 붙인다면 testset7->8 근데 여기 이해가 안돼.. 48 뭐임? 하루치?
testset9

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),...,SH(t-3),sin_time(t-3),cos_time(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
288,0.512282,0.274074,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.500000,1.000000,0.000000,0.288665,0.315278,0.139784,0.000000,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.565263,0.995722,0.000000,0.288665,0.315278,0.139784,0.000000,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,...,0.166667,0.308658,0.961940,0.000000,0.404037,0.425313,0.086034,0.000000,0.521805,0.144444,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.166667,0.370590,0.982963,0.000000,0.404037,0.425313,0.086034,0.000000,0.512174,0.148148,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.166667,0.434737,0.995722,0.000000,0.404037,0.425313,0.086034,0.000000
289,0.507845,0.275926,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.565263,0.995722,0.000000,0.288665,0.315278,0.139784,0.000000,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.629410,0.982963,0.000000,0.288665,0.315278,0.139784,0.000000,0.522779,0.281481,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,...,0.166667,0.370590,0.982963,0.000000,0.404037,0.425313,0.086034,0.000000,0.512174,0.148148,0.093368,0.180435,0.091892,0.758337,0.928639,0.086222,0.166667,0.434737,0.995722,0.000000,0.404037,0.425313,0.086034,0.000000,0.484904,0.153704,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.166667,0.500000,1.000000,0.000000,0.460685,0.437840,0.150545,0.000000
290,0.532626,0.277778,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.629410,0.982963,0.000000,0.288665,0.315278,0.139784,0.000000,0.522779,0.281481,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.691342,0.961940,0.000000,0.288665,0.315278,0.139784,0.000000,0.550698,0.283333,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,...,0.166667,0.434737,0.995722,0.000000,0.404037,0.425313,0.086034,0.000000,0.484904,0.153704,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.166667,0.500000,1.000000,0.000000,0.460685,0.437840,0.150545,0.000000,0.476031,0.157407,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.166667,0.565263,0.995722,0.000000,0.460685,0.437840,0.150545,0.000000
291,0.522779,0.281481,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.691342,0.961940,0.000000,0.288665,0.315278,0.139784,0.000000,0.550698,0.283333,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,0.166667,0.750000,0.933013,0.000000,0.288665,0.315278,0.139784,0.000000,0.546045,0.285185,0.343551,0.395652,0.386486,0.510941,0.753326,0.278025,...,0.166667,0.500000,1.000000,0.000000,0.460685,0.437840,0.150545,0.000000,0.476031,0.157407,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.166667,0.565263,0.995722,0.000000,0.460685,0.437840,0.150545,0.000000,0.452116,0.161111,0.260104,0.336957,0.224324,0.910587,0.951175,0.308772,0.166667,0.629410,0.982963,0.000000,0.460685,0.437840,0.150545,0.000000
292,0.550698,0.283333,0

In [38]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'

    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season', 'T' : 'Temp'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'Temp', 'TARGET', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHIs'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset5 = add_variables(testset5)
    testset6 = testset5[['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'SH', 'sin_time', 'cos_time', 'GHIs', 'Avg_TAR', 'Max_TAR', 'Min_TAR', 'TARGET']] #'DHI', 'DNI', 'season'
    testset6 = pd.DataFrame(scaler.transform(testset6), columns=testset6.columns)
    testset7 = test_to_supervised(testset6, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-144),Temp(t-144),Avg_temp(t-144),Max_temp(t-144),Min_temp(t-144),Avg_GHI(t-144),Max_GHI(t-144),Min_GHI(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHIs(t-144),Avg_TAR(t-144),Max_TAR(t-144),Min_TAR(t-144),TARGET(t-144),RH(t-143),Temp(t-143),Avg_temp(t-143),Max_temp(t-143),Min_temp(t-143),Avg_GHI(t-143),Max_GHI(t-143),Min_GHI(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHIs(t-143),Avg_TAR(t-143),Max_TAR(t-143),Min_TAR(t-143),TARGET(t-143),RH(t-142),Temp(t-142),Avg_temp(t-142),Max_temp(t-142),Min_temp(t-142),Avg_GHI(t-142),Max_GHI(t-142),Min_GHI(t-142),...,SH(t-3),sin_time(t-3),cos_time(t-3),GHIs(t-3),Avg_TAR(t-3),Max_TAR(t-3),Min_TAR(t-3),TARGET(t-3),RH(t-2),Temp(t-2),Avg_temp(t-2),Max_temp(t-2),Min_temp(t-2),Avg_GHI(t-2),Max_GHI(t-2),Min_GHI(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHIs(t-2),Avg_TAR(t-2),Max_TAR(t-2),Min_TAR(t-2),TARGET(t-2),RH(t-1),Temp(t-1),Avg_temp(t-1),Max_temp(t-1),Min_temp(t-1),Avg_GHI(t-1),Max_GHI(t-1),Min_GHI(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHIs(t-1),Avg_TAR(t-1),Max_TAR(t-1),Min_TAR(t-1),TARGET(t-1)
288,0.557299,0.353704,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.500000,1.000000,0.0,0.260408,0.389642,0.032259,0.0,0.580132,0.344444,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.565263,0.995722,0.0,0.260408,0.389642,0.032259,0.0,0.629477,0.337037,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,...,0.166667,0.308658,0.961940,0.0,0.314719,0.369770,6.451845e-02,0.0,0.798507,0.303704,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.166667,0.370590,0.982963,0.0,0.314719,0.369770,6.451845e-02,0.0,0.805108,0.301852,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.166667,0.434737,0.995722,0.0,0.314719,0.369770,6.451845e-02,0.0
289,0.580132,0.344444,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.565263,0.995722,0.0,0.260408,0.389642,0.032259,0.0,0.629477,0.337037,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.629410,0.982963,0.0,0.260408,0.389642,0.032259,0.0,0.634672,0.335185,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,...,0.166667,0.370590,0.982963,0.0,0.314719,0.369770,6.451845e-02,0.0,0.805108,0.301852,0.308146,0.317391,0.362162,0.568941,0.833437,0.045658,0.166667,0.434737,0.995722,0.0,0.314719,0.369770,6.451845e-02,0.0,0.793529,0.300000,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.166667,0.500000,1.000000,0.0,0.068938,0.062865,4.301463e-02,0.0
290,0.629477,0.337037,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.629410,0.982963,0.0,0.260408,0.389642,0.032259,0.0,0.634672,0.335185,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.691342,0.961940,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,...,0.166667,0.434737,0.995722,0.0,0.314719,0.369770,6.451845e-02,0.0,0.793529,0.300000,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.166667,0.500000,1.000000,0.0,0.068938,0.062865,4.301463e-02,0.0,0.800130,0.298148,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.166667,0.565263,0.995722,0.0,0.068938,0.062865,4.301463e-02,0.0
291,0.634672,0.335185,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.691342,0.961940,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.750000,0.933013,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,...,0.166667,0.500000,1.000000,0.0,0.068938,0.062865,4.301463e-02,0.0,0.800130,0.298148,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.166667,0.565263,0.995722,0.0,0.068938,0.062865,4.301463e-02,0.0,0.782599,0.296296,0.215979,0.178261,0.286486,0.065130,0.095574,0.010807,0.166667,0.629410,0.982963,0.0,0.068938,0.062865,4.301463e-02,0.0
292,0.672113,0.333333,0.366371,0.391304,0.354054,0.449593,0.868530,0.008105,0.166667,0.750000,0.933013,0.0,0.260408,0.389642,0.032259,0.0,0.672113,0.333333,0.366371,0.391304,0

In [39]:
Y_train1 = pd.DataFrame(Y_train.iloc[:,0])
Y_train2 = pd.DataFrame(Y_train.iloc[:,1])

Y_valid1 = pd.DataFrame(Y_valid.iloc[:,0])
Y_valid2 = pd.DataFrame(Y_valid.iloc[:,1])

In [40]:
[['RH', 'Temp','Avg_temp', 'Max_temp', 'Min_temp', 'Avg_GHI', 'Max_GHI','Min_GHI', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]

RH_train = X_train.filter(regex='RH')
T_train = X_train.filter(regex='Temp')
# SH_train = X_train.filter(regex='SH')
SH_train = X_train.filter(regex='SH').iloc[:,[0,48,96]]
Avg_temp_train = X_train.filter(regex='Avg_temp').iloc[:,[0,48,96]]
Max_temp_train = X_train.filter(regex='Max_temp').iloc[:,[0,48,96]]
Min_temp_train = X_train.filter(regex='Min_temp').iloc[:,[0,48,96]]
Avg_GHI_train = X_train.filter(regex='Avg_GHI').iloc[:,[0,48,96]]
Max_GHI_train = X_train.filter(regex='Max_GHI').iloc[:,[0,48,96]]
Min_GHI_train = X_train.filter(regex='Min_GHI').iloc[:,[0,48,96]]
Avg_TAR_train = X_train.filter(regex='Avg_TAR').iloc[:,[0,48,96]]
Max_TAR_train = X_train.filter(regex='Max_TAR').iloc[:,[0,48,96]]
Min_TAR_train = X_train.filter(regex='Min_TAR').iloc[:,[0,48,96]]
sin_train = X_train.filter(regex='sin_time').iloc[:,-6:]
cos_train = X_train.filter(regex='cos_time').iloc[:,-6:]
GHI_train = X_train.filter(regex='GHIs')
TARGET_train = X_train.filter(regex='TARGET')



RH_valid = X_valid.filter(regex='RH')
T_valid = X_valid.filter(regex='Temp')
# SH_valid = X_valid.filter(regex='SH')
SH_valid = X_valid.filter(regex='SH').iloc[:,[0,48,96]]
Avg_temp_valid = X_valid.filter(regex='Avg_temp').iloc[:,[0,48,96]]
Max_temp_valid = X_valid.filter(regex='Max_temp').iloc[:,[0,48,96]]
Min_temp_valid = X_valid.filter(regex='Min_temp').iloc[:,[0,48,96]]
Avg_GHI_valid = X_valid.filter(regex='Avg_GHI').iloc[:,[0,48,96]]
Max_GHI_valid = X_valid.filter(regex='Max_GHI').iloc[:,[0,48,96]]
Min_GHI_valid = X_valid.filter(regex='Min_GHI').iloc[:,[0,48,96]]
Avg_TAR_valid = X_valid.filter(regex='Avg_TAR').iloc[:,[0,48,96]]
Max_TAR_valid = X_valid.filter(regex='Max_TAR').iloc[:,[0,48,96]]
Min_TAR_valid = X_valid.filter(regex='Min_TAR').iloc[:,[0,48,96]]
sin_valid = X_valid.filter(regex='sin_time').iloc[:,-6:]
cos_valid = X_valid.filter(regex='cos_time').iloc[:,-6:]
GHI_valid = X_valid.filter(regex='GHIs')
TARGET_valid = X_valid.filter(regex='TARGET')

RH_test = X_test.filter(regex='RH')
T_test = X_test.filter(regex='Temp')
# SH_test = X_test.filter(regex='SH')
SH_test = X_test.filter(regex='SH').iloc[:,[0,48,96]]
Avg_temp_test = X_test.filter(regex='Avg_temp').iloc[:,[0,48,96]]
Max_temp_test = X_test.filter(regex='Max_temp').iloc[:,[0,48,96]]
Min_temp_test = X_test.filter(regex='Min_temp').iloc[:,[0,48,96]]
Avg_GHI_test = X_test.filter(regex='Avg_GHI').iloc[:,[0,48,96]]
Max_GHI_test = X_test.filter(regex='Max_GHI').iloc[:,[0,48,96]]
Min_GHI_test = X_test.filter(regex='Min_GHI').iloc[:,[0,48,96]]
Avg_TAR_test = X_test.filter(regex='Avg_TAR').iloc[:,[0,48,96]]
Max_TAR_test = X_test.filter(regex='Max_TAR').iloc[:,[0,48,96]]
Min_TAR_test = X_test.filter(regex='Min_TAR').iloc[:,[0,48,96]]
sin_test = X_test.filter(regex='sin_time').iloc[:,-6:]
cos_test = X_test.filter(regex='cos_time').iloc[:,-6:]
GHI_test = X_test.filter(regex='GHIs')
TARGET_test = X_test.filter(regex='TARGET')

In [41]:
GHI_train

,GHIs(t-144),GHIs(t-143),GHIs(t-142),GHIs(t-141),GHIs(t-140),GHIs(t-139),GHIs(t-138),GHIs(t-137),GHIs(t-136),GHIs(t-135),GHIs(t-134),GHIs(t-133),GHIs(t-132),GHIs(t-131),GHIs(t-130),GHIs(t-129),GHIs(t-128),GHIs(t-127),GHIs(t-126),GHIs(t-125),GHIs(t-124),GHIs(t-123),GHIs(t-122),GHIs(t-121),GHIs(t-120),GHIs(t-119),GHIs(t-118),GHIs(t-117),GHIs(t-116),GHIs(t-115),GHIs(t-114),GHIs(t-113),GHIs(t-112),GHIs(t-111),GHIs(t-110),GHIs(t-109),GHIs(t-108),GHIs(t-107),GHIs(t-106),GHIs(t-105),...,GHIs(t-40),GHIs(t-39),GHIs(t-38),GHIs(t-37),GHIs(t-36),GHIs(t-35),GHIs(t-34),GHIs(t-33),GHIs(t-32),GHIs(t-31),GHIs(t-30),GHIs(t-29),GHIs(t-28),GHIs(t-27),GHIs(t-26),GHIs(t-25),GHIs(t-24),GHIs(t-23),GHIs(t-22),GHIs(t-21),GHIs(t-20),GHIs(t-19),GHIs(t-18),GHIs(t-17),GHIs(t-16),GHIs(t-15),GHIs(t-14),GHIs(t-13),GHIs(t-12),GHIs(t-11),GHIs(t-10),GHIs(t-9),GHIs(t-8),GHIs(t-7),GHIs(t-6),GHIs(t-5),GHIs(t-4),GHIs(t-3),GHIs(t-2),GHIs(t-1)
1555,0.224206,0.270798,0.247512,0.084251,0.354219,0.219664,0.289061,0.325244,0.316796,0.200954,0.132583,0.119692,0.067401,0.034587,0.017737,0.004434,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.190399,0.179289,0.116178,0.098441,0.099173,0.119570,0.069560,0.014190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013303,0.043456,0.031040,0.066514
38121,0.000000,0.000000,0.000000,0.000000,0.000000,0.181480,0.332636,0.448880,0.548533,0.634516,0.709885,0.772131,0.870022,0.917264,0.949945,0.970909,0.977860,0.972283,0.953479,0.919441,0.874510,0.816596,0.744720,0.657896,0.552619,0.418367,0.231930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.513298,0.604165,0.627928,0.732476,0.683357,0.507460,0.709102,0.839957,0.526038,0.266649,0.119666,0.226212,0.098441,0.509726,0.470415,0.394260,0.318655,0.227952,0.114885,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4783,0.631380,0.516900,0.566730,0.460882,0.179424,0.154470,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039553,0.118144,0.208273,0.251161,0.497686,0.628705,0.694881,0.753416,0.802807,0.844298,0.875242,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.114023,0.248962,0.382357,0.454437,0.501480,0.578219,0.683300,0.741637,0.790839,0.831895,0.840049,0.891245,0.900273,0.901828,0.503765,0.331133,0.143013,0.083364,0.210464
20757,0.717651,0.550731,0.540988,0.660939,0.462158,0.389180,0.091346,0.015077,0.013303,0.207091,0.256671,0.208230,0.155392,0.098447,0.035956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.635482,0.594970,0.573904,0.488159,0.397092,0.289450,0.155602,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.245946,0.380618,0.487134,0.581779,0.661138,0.732809,0.790806
47626,0.000000,0.000000,0.018624,0.212936,0.334909,0.433005,0.519514,0.592773,0.657811,0.714540,0.761506,0.340212,0.833616,0.792192,0.612168,0.797266,0.794330,0.02217

In [42]:
trainset = pd.concat([RH_train, T_train, sin_train, cos_train, GHI_train, TARGET_train, SH_train, Avg_GHI_train, Avg_temp_train, Max_GHI_train, Max_temp_train, Min_GHI_train, Min_temp_train, Avg_TAR_train, Max_TAR_train, Min_TAR_train], axis = 1)
validset = pd.concat([RH_valid, T_valid, sin_valid, cos_valid, GHI_valid, TARGET_valid, SH_valid, Avg_GHI_valid, Avg_temp_valid, Max_GHI_valid, Max_temp_valid, Min_GHI_valid, Min_temp_valid, Avg_TAR_valid, Max_TAR_valid, Min_TAR_valid], axis = 1)
testset = pd.concat([RH_test, T_test, sin_test, cos_test, GHI_test, TARGET_test, SH_test, Avg_GHI_test, Avg_temp_test, Max_GHI_test,Max_temp_test, Min_GHI_test, Min_temp_test, Avg_TAR_test, Max_TAR_test, Min_TAR_test], axis = 1)

In [43]:
trainset

,RH(t-144),RH(t-143),RH(t-142),RH(t-141),RH(t-140),RH(t-139),RH(t-138),RH(t-137),RH(t-136),RH(t-135),RH(t-134),RH(t-133),RH(t-132),RH(t-131),RH(t-130),RH(t-129),RH(t-128),RH(t-127),RH(t-126),RH(t-125),RH(t-124),RH(t-123),RH(t-122),RH(t-121),RH(t-120),RH(t-119),RH(t-118),RH(t-117),RH(t-116),RH(t-115),RH(t-114),RH(t-113),RH(t-112),RH(t-111),RH(t-110),RH(t-109),RH(t-108),RH(t-107),RH(t-106),RH(t-105),...,TARGET(t-10),TARGET(t-9),TARGET(t-8),TARGET(t-7),TARGET(t-6),TARGET(t-5),TARGET(t-4),TARGET(t-3),TARGET(t-2),TARGET(t-1),SH(t-144),SH(t-96),SH(t-48),Avg_GHI(t-144),Avg_GHI(t-96),Avg_GHI(t-48),Avg_temp(t-144),Avg_temp(t-96),Avg_temp(t-48),Max_GHI(t-144),Max_GHI(t-96),Max_GHI(t-48),Max_temp(t-144),Max_temp(t-96),Max_temp(t-48),Min_GHI(t-144),Min_GHI(t-96),Min_GHI(t-48),Min_temp(t-144),Min_temp(t-96),Min_temp(t-48),Avg_TAR(t-144),Avg_TAR(t-96),Avg_TAR(t-48),Max_TAR(t-144),Max_TAR(t-96),Max_TAR(t-48),Min_TAR(t-144),Min_TAR(t-96),Min_TAR(t-48)
1555,0.595282,0.558381,0.558056,0.564225,0.563792,0.568986,0.568661,0.574397,0.574288,0.583054,0.631425,0.707391,0.765934,0.777080,0.841359,0.804134,0.804134,0.836165,0.836057,0.877286,0.877178,0.860080,0.859755,0.838546,0.838221,0.797966,0.797641,0.765285,0.764852,0.742344,0.742019,0.780543,0.780219,0.757927,0.757602,0.734120,0.733687,0.728601,0.728384,0.788551,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014089,0.046025,0.032874,0.070444,0.250000,0.250000,0.250000,0.186493,0.075254,0.203323,0.396867,0.340470,0.256397,0.284410,0.127587,0.439039,0.369565,0.326087,0.239130,0.010807,0.010807,0.040526,0.513514,0.405405,0.324324,0.197800,0.088084,0.205981,0.234625,0.100559,0.324734,0.043011,0.043011,0.161295
38121,0.543339,0.529380,0.529272,0.518883,0.435234,0.434585,0.398009,0.386971,0.324207,0.272157,0.248242,0.224326,0.224326,0.209393,0.209285,0.199978,0.199978,0.189915,0.189915,0.199762,0.219349,0.247159,0.270317,0.412509,0.486203,0.424954,0.462612,0.436425,0.436425,0.411319,0.411319,0.427335,0.427335,0.442701,0.442593,0.417812,0.417704,0.431555,0.431339,0.448328,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.416667,0.416667,0.416667,0.924369,0.917638,0.538584,0.411488,0.465274,0.492428,0.975467,0.954153,0.822656,0.456522,0.543478,0.521739,0.550169,0.596907,0.297193,0.432432,0.432432,0.513514,0.752896,0.754856,0.498098,0.719555,0.710101,0.648315,0.526842,0.548339,0.430056
4783,0.114273,0.118277,0.131696,0.247376,0.320203,0.411103,0.484255,0.518883,0.562818,0.572557,0.620820,0.607943,0.607835,0.589763,0.639433,0.622876,0.675360,0.660318,0.660102,0.645926,0.645818,0.632291,0.632074,0.622335,0.622119,0.615626,0.615410,0.617682,0.617574,0.659994,0.561195,0.450601,0.381777,0.271724,0.248566,0.192836,0.175089,0.147170,0.132561,0.116870,...,0.808602,0.829254,0.875271,0.881845,0.873393,0.519340,0.347479,0.150261,0.088279,0.219762,0.666667,0.666667,0.666667,0.711510,0.479534,0.425246,0.560836,0.537859,0.443864,0.902900,0.861756,0.839307,0.608696,0.630435,0.478261,0.112791,0.013509,0.008105,0.540541,0.513514,0.459459,0.781823,0.573040,0.472855,0.867162,0.832601,0.739446,0.279550,0.053759,0.032259
20757,0.312520,0.286116,0.285900,0.285900,0.310356,0.311546,0.338059,0.341089,0.369873,0.376474,0.407965,0.486311,0.525701,0.593118,0.640732,0.650254,0.650146,0.641273,0.692890,0.671789,0.726112,0.705768,0.763013,0.750243,0.811059,0.799264,0.864409,0.842441,0.911481,0.875446,0.875122,0.811059,0.810735,0.727843,0.727627,0.716697,0.716481,0.673304,0.673087,0.670057,...,0.000000,0.000000,0.000000,0.085475,0.182218,0.280835,0.378512,0.467734,0.548501,0.616118,0.416667,0.416667,0.416667,0.451991,0.371983,0.691717,0.546214,0.431854,0.374935,0.744550,0.548545,0.909213,0.586957,0.413043,0.369565,0.037825,0.078351,0.471333,0.540541,0.459459,0.405405,0.438110,0.314614,0.647749,0.540421,0.370782,0.693405,0.150510,0.311788,0.548358
47626,0.951304,0.881615,0.818743,0.758792,0.665404,0.571800,0.468997,0.402013,0.319013,0.294449,0.263067,0.262959,0.243372,0.243264,

In [44]:
validset

,RH(t-144),RH(t-143),RH(t-142),RH(t-141),RH(t-140),RH(t-139),RH(t-138),RH(t-137),RH(t-136),RH(t-135),RH(t-134),RH(t-133),RH(t-132),RH(t-131),RH(t-130),RH(t-129),RH(t-128),RH(t-127),RH(t-126),RH(t-125),RH(t-124),RH(t-123),RH(t-122),RH(t-121),RH(t-120),RH(t-119),RH(t-118),RH(t-117),RH(t-116),RH(t-115),RH(t-114),RH(t-113),RH(t-112),RH(t-111),RH(t-110),RH(t-109),RH(t-108),RH(t-107),RH(t-106),RH(t-105),...,TARGET(t-10),TARGET(t-9),TARGET(t-8),TARGET(t-7),TARGET(t-6),TARGET(t-5),TARGET(t-4),TARGET(t-3),TARGET(t-2),TARGET(t-1),SH(t-144),SH(t-96),SH(t-48),Avg_GHI(t-144),Avg_GHI(t-96),Avg_GHI(t-48),Avg_temp(t-144),Avg_temp(t-96),Avg_temp(t-48),Max_GHI(t-144),Max_GHI(t-96),Max_GHI(t-48),Max_temp(t-144),Max_temp(t-96),Max_temp(t-48),Min_GHI(t-144),Min_GHI(t-96),Min_GHI(t-48),Min_temp(t-144),Min_temp(t-96),Min_temp(t-48),Avg_TAR(t-144),Avg_TAR(t-96),Avg_TAR(t-48),Max_TAR(t-144),Max_TAR(t-96),Max_TAR(t-48),Min_TAR(t-144),Min_TAR(t-96),Min_TAR(t-48)
29851,0.407532,0.399199,0.399199,0.410562,0.443891,0.481982,0.520290,0.573315,0.618223,0.676875,0.729358,0.761931,0.820799,0.823179,0.823288,0.810410,0.752300,0.697760,0.647874,0.554702,0.477654,0.334488,0.309057,0.261876,0.241099,0.213180,0.213072,0.191105,0.190997,0.171951,0.171951,0.173683,0.189590,0.194027,0.211449,0.240017,0.260686,0.301050,0.326155,0.389785,...,0.204720,0.130534,0.061042,0.008452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.666667,0.666667,0.515057,0.583855,0.171930,0.822454,0.753003,0.701305,0.877241,0.827008,0.447036,0.869565,0.782609,0.739130,0.024316,0.029719,0.018912,0.783784,0.756757,0.702703,0.603234,0.624338,0.174166,0.810485,0.764458,0.254469,9.675108e-02,0.118255,7.526042e-02
9985,0.694081,0.692025,0.744184,0.738448,0.738556,0.723298,0.778379,0.759658,0.706201,0.652310,0.606320,0.542582,0.468564,0.387620,0.359593,0.282221,0.260902,0.233741,0.215561,0.194892,0.194892,0.176280,0.176171,0.160589,0.160480,0.160156,0.160048,0.159507,0.173791,0.172709,0.187967,0.186560,0.202792,0.214587,0.252029,0.389352,0.450817,0.483497,0.520182,0.506222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.916667,0.916667,0.916667,0.693523,0.592433,0.521409,0.901828,0.877285,0.924804,0.788960,0.684787,0.768938,0.913043,0.869565,0.913043,0.091860,0.248562,0.294631,0.864865,0.891892,0.891892,0.853802,0.684679,0.644212,0.935088,0.765456,0.885860,2.902421e-01,0.268752,2.579930e-01
29105,0.276485,0.244779,0.225192,0.201277,0.201169,0.181366,0.181257,0.164268,0.164268,0.165458,0.165350,0.167623,0.183097,0.182556,0.199113,0.198355,0.215994,0.240450,0.282870,0.376583,0.406450,0.406233,0.438589,0.424197,0.457851,0.451466,0.487285,0.485337,0.523753,0.515096,0.515096,0.506655,0.506655,0.496808,0.536306,0.526458,0.526458,0.521589,0.521697,0.526350,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.057288,0.141809,0.237595,0.334320,0.430099,0.666667,0.666667,0.666667,0.629333,0.422714,0.398217,0.796867,0.808877,0.765013,0.806710,0.819663,0.818247,0.804348,0.804348,0.695652,0.197228,0.027018,0.107654,0.756757,0.837838,0.837838,0.702731,0.507624,0.509196,0.818904,0.832507,0.826276,7.095438e-01,0.107495,3.655107e-01
43751,0.121848,0.116979,0.116979,0.114165,0.114057,0.112217,0.112109,0.114923,0.126718,0.133535,0.146521,0.172600,0.188291,0.258522,0.302673,0.412834,0.444757,0.428741,0.461855,0.504924,0.543664,0.559139,0.601775,0.601017,0.646900,0.630884,0.679039,0.678498,0.678498,0.695920,0.749053,0.769614,0.828374,0.824370,0.766259,0.699383,0.603939,0.492263,0.456985,0.364787,...,0.281733,0.378454,0.474236,0.567192,0.655457,0.735268,0.806626,0.867653,0.915533,0.952144,1.000000,1.000000,1.000000,0.410529,0.556023,0.501363,0.891906,0.848042,0.775457,0.701097,0.700832,0.688959,0.891304,0.869565,0.760870,0.000000,0.107346,0.240599,0.864865,0.837838,0.783784,0.537000,0.743023,0.620522,0.827230,0.919403,0.849284,2.363586e-07,0.301006,2.902785e-01
4351,0.140461,0.145547,0.177254,0.305595,0.361000,0.379180,0.412077,0.475057,0.515312,0.525160,0.56

In [45]:
testset

,RH(t-144),RH(t-143),RH(t-142),RH(t-141),RH(t-140),RH(t-139),RH(t-138),RH(t-137),RH(t-136),RH(t-135),RH(t-134),RH(t-133),RH(t-132),RH(t-131),RH(t-130),RH(t-129),RH(t-128),RH(t-127),RH(t-126),RH(t-125),RH(t-124),RH(t-123),RH(t-122),RH(t-121),RH(t-120),RH(t-119),RH(t-118),RH(t-117),RH(t-116),RH(t-115),RH(t-114),RH(t-113),RH(t-112),RH(t-111),RH(t-110),RH(t-109),RH(t-108),RH(t-107),RH(t-106),RH(t-105),...,TARGET(t-10),TARGET(t-9),TARGET(t-8),TARGET(t-7),TARGET(t-6),TARGET(t-5),TARGET(t-4),TARGET(t-3),TARGET(t-2),TARGET(t-1),SH(t-144),SH(t-96),SH(t-48),Avg_GHI(t-144),Avg_GHI(t-96),Avg_GHI(t-48),Avg_temp(t-144),Avg_temp(t-96),Avg_temp(t-48),Max_GHI(t-144),Max_GHI(t-96),Max_GHI(t-48),Max_temp(t-144),Max_temp(t-96),Max_temp(t-48),Min_GHI(t-144),Min_GHI(t-96),Min_GHI(t-48),Min_temp(t-144),Min_temp(t-96),Min_temp(t-48),Avg_TAR(t-144),Avg_TAR(t-96),Avg_TAR(t-48),Max_TAR(t-144),Max_TAR(t-96),Max_TAR(t-48),Min_TAR(t-144),Min_TAR(t-96),Min_TAR(t-48)
288,0.557299,0.580132,0.629477,0.634672,0.672113,0.672113,0.655557,0.650254,0.613678,0.609458,0.581647,0.576886,0.560437,0.560437,0.561303,0.516286,0.515529,0.470512,0.462288,0.425495,0.412509,0.392490,0.366627,0.363489,0.342063,0.347906,0.361108,0.376691,0.412401,0.444324,0.463586,0.512607,0.515637,0.515637,0.547668,0.575803,0.566389,0.591494,0.586300,0.596364,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.449593,0.861974,0.568941,0.366371,0.296919,0.308146,0.868530,0.916166,0.833437,0.391304,0.326087,0.317391,0.008105,0.276417,0.045658,0.354054,0.348649,0.362162,0.260408,0.430127,0.314719,0.389642,0.412704,0.369770,0.032259,0.161290,0.064518
289,0.580132,0.629477,0.634672,0.672113,0.672113,0.655557,0.650254,0.613678,0.609458,0.581647,0.576886,0.560437,0.560437,0.561303,0.516286,0.515529,0.470512,0.462288,0.425495,0.412509,0.392490,0.366627,0.363489,0.342063,0.347906,0.361108,0.376691,0.412401,0.444324,0.463586,0.512607,0.515637,0.515637,0.547668,0.575803,0.566389,0.591494,0.586300,0.596364,0.594092,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.449593,0.861974,0.568941,0.366371,0.296919,0.308146,0.868530,0.916166,0.833437,0.391304,0.326087,0.317391,0.008105,0.276417,0.045658,0.354054,0.348649,0.362162,0.260408,0.430127,0.314719,0.389642,0.412704,0.369770,0.032259,0.161290,0.064518
290,0.629477,0.634672,0.672113,0.672113,0.655557,0.650254,0.613678,0.609458,0.581647,0.576886,0.560437,0.560437,0.561303,0.516286,0.515529,0.470512,0.462288,0.425495,0.412509,0.392490,0.366627,0.363489,0.342063,0.347906,0.361108,0.376691,0.412401,0.444324,0.463586,0.512607,0.515637,0.515637,0.547668,0.575803,0.566389,0.591494,0.586300,0.596364,0.594092,0.610324,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.449593,0.861974,0.568941,0.366371,0.296919,0.308146,0.868530,0.916166,0.833437,0.391304,0.326087,0.317391,0.008105,0.276417,0.045658,0.354054,0.348649,0.362162,0.260408,0.430127,0.314719,0.389642,0.412704,0.369770,0.032259,0.161290,0.064518
291,0.634672,0.672113,0.672113,0.655557,0.650254,0.613678,0.609458,0.581647,0.576886,0.560437,0.560437,0.561303,0.516286,0.515529,0.470512,0.462288,0.425495,0.412509,0.392490,0.366627,0.363489,0.342063,0.347906,0.361108,0.376691,0.412401,0.444324,0.463586,0.512607,0.515637,0.515637,0.547668,0.575803,0.566389,0.591494,0.586300,0.596364,0.594092,0.610324,0.600909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.166667,0.166667,0.449593,0.861974,0.568941,0.366371,0.296919,0.308146,0.868530,0.916166,0.833437,0.391304,0.326087,0.317391,0.008105,0.276417,0.045658,0.354054,0.348649,0.362162,0.260408,0.430127,0.314719,0.389642,0.412704,0.369770,0.032259,0.161290,0.064518
292,0.672113,0.672113,0.655557,0.650254,0.613678,0.609458,0.581647,0.576886,0.560437,0.560437,0.561303,0.516286,0.515529,0.470512,0.462288,0.425495,0.412509,0.392490,0.366627,0.363489,0.342063,0.347

In [50]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=15000, bagging_fraction=0.7, learning_rate=0.05, save_binary = True, subsample=0.7, device = 'gpu', min_data_in_leaf = 30)          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [51]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, Y_train, X_valid, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [52]:
qq = 0
quantiles = [0.1-qq, 0.2-qq, 0.3-qq, 0.4-qq, 0.5, 0.6-qq, 0.7-qq, 0.8-qq, 0.9-qq]

In [ ]:
tick = time.time()

# Target1

models_1, results_1 = train_data(trainset, Y_train1, validset, Y_valid1, testset)
results_1.sort_index()[:48]

0.1
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.25484
[1000]	valid_0's quantile: 1.23691
[1500]	valid_0's quantile: 1.22938
[2000]	valid_0's quantile: 1.22483
[2500]	valid_0's quantile: 1.22224
[3000]	valid_0's quantile: 1.22094
[3500]	valid_0's quantile: 1.21963
[4000]	valid_0's quantile: 1.21894
[4500]	valid_0's quantile: 1.21883
[5000]	valid_0's quantile: 1.21762
[5500]	valid_0's quantile: 1.21759
[6000]	valid_0's quantile: 1.21679


In [ ]:
# Target2

models_2, results_2 = train_data(trainset, Y_train2, validset, Y_valid2, testset)
tock = time.time()
results_2.sort_index()

In [ ]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

In [ ]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [ ]:
submission3.to_csv('submission_mulvar2.csv')
!cp submission_mulvar2.csv "drive/My Drive/"